In [46]:
## We are using the 'colorability' db credentials for all association ratings experiments. 
## Don't get confused. Yes, that means you Kushin.

##make sure you're connected to UW VPN

In [1]:
import os
import mysql.connector
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from scipy import stats


In [321]:
# db = mysql.connector.connect(
#    host='127.0.0.1',
#     port= 3306,
#    user='root',
#    passwd='kushinm11',
#     db = 'db1',
#     auth_plugin='mysql_native_password'
# )
# table_name = ''

## Connect to DB

In [53]:
db = mysql.connector.connect(
   host="schlosslab-exp.wid.wisc.edu",
    port= 3306,
   user="colorability_op",
   passwd="auy2fovei7Bee6uC",
    db = 'colorability'
)


In [54]:
##get list of table names

cursor = db.cursor()

cursor.execute("SHOW TABLES")    # execute 'SHOW TABLES' (but data is not returned)
for (table_name,) in cursor:
    print(table_name)

condition_log
data_log
register_log


In [12]:
# db = mysql.connector.connect(
#    host="kclab-exp.wid.wisc.edu",
#     port= 3306,
#    user="color_concept_rate_op",
#    passwd="eeve8yai8pohZoh9",
#     db = 'color_concept_rate'
# )
# table_name = 'register_log'

In [13]:
table_name = 'data_log'
df = pd.read_sql_query("SELECT * FROM {}".format(table_name),
    db) ### convert table into dataframe

In [45]:
## How many in each category?
df[df.trial_type=='image-slider-responseMAS'].groupby('concept')['subject_id'].apply(lambda x: len(list(np.unique(x))))


concept
animals        16
automobiles    18
values         19
weather        18
Name: subject_id, dtype: int64

In [20]:
print('How many subjects so far?')
print(df.workerID.nunique())


How many subjects so far?
74


In [27]:
df[(df.subject_id=='a17uet7exuj4ldasvvdh') &(df.trial_type=='image-slider-responseMAS')]

,responses,question_order,trial_type,trial_index,rt,time_elapsed,condition,response,prompt,color_rgb,concept,time_finished,rowID,color_index,concept_category,subject_id,workerID
9,None,None,image-slider-responseMAS,9,8855.44,80373.0,None,10.0,love,"[100, 53, 97, 57, 101, 52]",values,1.612350e+12,106554,16.0,None,a17uet7exuj4ldasvvdh,70360
10,None,None,image-slider-responseMAS,10,5457.42,86834.0,None,-110.0,love,"[56, 53, 53, 56, 102, 52]",values,1.612350e+12,106555,2.0,None,a17uet7exuj4ldasvvdh,70360
11,None,None,image-slider-responseMAS,11,2971.40,90809.0,None,-153.0,love,"[54, 55, 99, 102, 53, 99]",values,1.612350e+12,106556,51.0,None,a17uet7exuj4ldasvvdh,70360
12,None,None,image-slider-responseMAS,12,2792.73,94603.0,None,41.0,love,"[101, 51, 49, 98, 55, 51]",values,1.612350e+12,106557,34.0,None,a17uet7exuj4ldasvvdh,70360
13,None,None,image-slider-responseMAS,13,2777.41,98382.0,None,-200.0,love,"[56, 51, 55, 54, 52, 99]",values,1.612350e+12,106558,42.0,None,a17uet7exuj4ldasvvdh,70360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,None,None,image-slider-responseMAS,367,2114.55,2393430.0,None,58.0,greed,"[57, 97, 102, 52, 52, 51]",values,1.612350e+12,106912,65.0,None,a17uet7exuj4ldasvvdh,70360
368,None,None,image-slider-responseMAS,368,3680.63,2398120.0,None,91.0,greed,"[49, 56, 52, 52, 49, 53]",values,1.612350e+12,106913,37.0,None,a17uet7exuj4ldasvvdh,70360
369,None,None,image-slider-responseMAS,369,3682.93,2402800.0,None,-109.0,greed,"[50, 101, 51, 48, 56, 54]",values,1.612350e+12,106914,5.0,None,a17uet7exuj4ldasvvdh,70360
370,None,None,image-slider-responseMAS,370,2920.90,2406730.0,None,-157.0,greed,"[102, 98, 97, 55, 49, 52]",values,1.612350e+12,106915,70.0,None,a17uet7exuj4ldasvvdh,70360
